In [1]:
import os
import cv2
from pathlib import Path
import random
import torch
import glob
from shutil import copyfile

In [2]:
#Change disk directory
base_path = Path("G:/Dissertation/")
if(Path().cwd() != Path(r"G:\Dissertation")):
    os.chdir(base_path)

In [3]:
#Define data_paths
# raw_data_path = Path("raw_data/Data examples/")
# raw_visibility_path = raw_data_path / Path("1_Visibility/")
# raw_quality_path = raw_data_path/ Path("2_Quality/")
dataset_path = Path('dataset/')

In [4]:
from data_preparation import split_data

data_dir = Path("small_split/")
split_size = [0.8,0.1,0.1]

#split_data(dataset_path, data_dir, split_size, num_img_class=1000)

In [5]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
from torchvision import transforms
#Create transform (in this case for the ResNet images are resized to 224x224 and transformed into Tensors)
data_transform = transforms.Compose([
  # Resize our images to 64x64
  transforms.Resize(size=(224, 224)),
  # Flip the images randomly on the horizontal (just to make it as independent from the position of the pipeline)
  transforms.RandomHorizontalFlip(p=0.5),
  transforms.RandomVerticalFlip(p=0.5),
  # Turn the image into a torch.Tensor
  transforms.ToTensor() 
])

In [25]:
from dataloaders import create_dataloaders

#Paths criados pelo split_data
train_dir = data_dir / Path('train/')
validation_dir = data_dir / Path('validation/')
test_dir = data_dir / Path('test/')

BATCH_SIZE = 128 # in the ResNet is 128 but my GPU doesn't have enough memory for that

train_dataloader, validation_dataloader, test_dataloader, class_names = create_dataloaders(train_dir,test_dir,validation_dir,data_transform, 32)

In [8]:
# Get a single image batch
image_batch, label_batch = next(iter(train_dataloader))
image_batch.shape, label_batch.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [26]:
torch.cuda.empty_cache()

In [24]:
!nvidia-smi

Thu Feb 16 11:31:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.01       Driver Version: 516.01       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   59C    P8     5W /  N/A |   2355MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
from engine import train
from ResNet50 import ResNet50
import torch.nn as nn

# Set number of epochs
NUM_EPOCHS = 5

model_1 = ResNet50().to(device)

# Setup loss function and optimizer 
loss_fn = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(params=model_1.parameters(),
                             lr=0.001)

# Start the timer
from timeit import default_timer as timer
start_time = timer() 

# Train model
model_results = train(model=model_1,
                    train_dataloader=train_dataloader,
                    validation_dataloader=validation_dataloader,
                    optimizer=optimizer,
                    loss_fn=loss_fn,
                    epochs=NUM_EPOCHS,
                    device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.7656 | train_acc: 0.6633 | validation_loss: 22.8928 | validation_acc: 0.2441
Epoch: 2 | train_loss: 0.5743 | train_acc: 0.7340 | validation_loss: 4.9895 | validation_acc: 0.4707
Epoch: 3 | train_loss: 0.5321 | train_acc: 0.7480 | validation_loss: 0.6341 | validation_acc: 0.7266
Epoch: 4 | train_loss: 0.5002 | train_acc: 0.7735 | validation_loss: 1.2028 | validation_acc: 0.5996
Epoch: 5 | train_loss: 0.4532 | train_acc: 0.7965 | validation_loss: 6.5499 | validation_acc: 0.3965
Total training time: 863.439 seconds


In [51]:
from utils import save_model, eval_model, accuracy_fn

models_path = Path('Models/')
model_name = 'ResNet50_1000_class_5_epoch.pth'

save_model(models_path, model_name, model_1)


model_path = models_path / model_name
loaded_model = ResNet50()
loaded_model.load_state_dict(torch.load(model_path))

Saving the model to: Models\ResNet50_1000_class_5_epoch.pth


<All keys matched successfully>

In [52]:
loaded_model.to(device)

loaded_model_results = eval_model(model= loaded_model,
                                  data_loader = validation_dataloader,
                                  loss_fn=loss_fn,
                                  accuracy_fn=accuracy_fn)

loaded_model_results

TypeError: 'module' object is not callable